In [ ]:
import os
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from reservoirpy.nodes import Reservoir

def random_weights(graph):
    random_assignment = np.random.random(graph.shape)
    return graph * random_assignment

def load_fly(return_tensor = False):
    pwd = os.getcwd()
    data_path = os.path.join(pwd, "science.add9330_data_s1_to_s4/Supplementary-Data-S1/all-all_connectivity_matrix.csv")
    # fly_mat = pd.read_csv('/Users/alexdavies/Projects/whatIsLarva/science.add9330_data_s1_to_s4/Supplementary-Data-S1/all-all_connectivity_matrix.csv').drop(columns=['Unnamed: 0'])
    fly_mat = pd.read_csv(
        data_path).drop(
        columns=['Unnamed: 0'])
    fly_mat = fly_mat.to_numpy()
    #

    # Could be fun to trim only to multiple-synapse connections?
    fly_mat[fly_mat > 1] = 1
    fly_mat[fly_mat != 1] = 0
    fly_mat[np.identity(fly_mat.shape[0], dtype=bool)] = 0.
    fly_graph = fly_mat
    #
    # random_assignment = np.random.random(fly_graph.shape)
    # fly_graph *= random_assignment

    if return_tensor:
        return torch.Tensor(fly_graph)
    else:
        return fly_graph

def create_random(fly_graph, return_tensor = False):
    rand_graph = nx.fast_gnp_random_graph(fly_graph.shape[0], np.sum(fly_graph) / (fly_graph.shape[0] ** 2))
    rand_graph = nx.to_numpy_array(rand_graph)
    #
    # random_assignment = np.random.random(rand_graph.shape)
    # rand_graph *= random_assignment

    if return_tensor:
        return torch.Tensor(rand_graph)
    else:
        return rand_graph

def objective(dataset, config, *, iss, N, sr, lr, ridge, seed):

    # This step may vary depending on what you put inside 'dataset'
    train_data, validation_data = dataset
    X_train, y_train = train_data
    X_val, y_val = validation_data

    # You can access anything you put in the config
    # file from the 'config' parameter.
    instances = config["instances_per_trial"]

    # The seed should be changed across the instances,
    # to be sure there is no bias in the results
    # due to initialization.
    variable_seed = seed

    losses = []; r2s = [];
    for n in range(instances):
        # Build your model given the input parameters
        reservoir = Reservoir(N,
                              sr=sr,
                              lr=lr,
                              inut_scaling=iss,
                              seed=variable_seed)

        readout = Ridge(ridge=ridge)

        model = reservoir >> readout


        # Train your model and test your model.
        predictions = model.fit(X_train, y_train) \
                           .run(X_test)

        loss = nrmse(y_test, predictions, norm_value=np.ptp(X_train))
        r2 = rsquare(y_test, predictions)

        # Change the seed between instances
        variable_seed += 1

        losses.append(loss)
        r2s.append(r2)

    # Return a dictionnary of metrics. The 'loss' key is mandatory when
    # using hyperopt.
    return {'loss': np.mean(losses),
            'r2': np.mean(r2s)}

def objective_given_graph(dataset, config, *, graph_to_use, iss, sr, lr, ridge, seed):

    # This step may vary depending on what you put inside 'dataset'
    train_data, validation_data = dataset
    X_train, y_train = train_data
    X_val, y_val = validation_data

    # You can access anything you put in the config
    # file from the 'config' parameter.
    instances = config["instances_per_trial"]

    # The seed should be changed across the instances,
    # to be sure there is no bias in the results
    # due to initialization.
    variable_seed = seed
    if graph_to_use == "fly":
        graph = load_fly()
    else:
        graph = create_random(load_fly())

    losses = []; r2s = [];
    for n in range(instances):
        # Build your model given the input parameters
        reservoir_graph = random_weights(graph)
        reservoir = Reservoir(W = reservoir_graph,
                              sr=sr,
                              lr=lr,
                              inut_scaling=iss,
                              seed=variable_seed)

        readout = Ridge(ridge=ridge)

        model = reservoir >> readout


        # Train your model and test your model.
        predictions = model.fit(X_train, y_train) \
                           .run(X_test)

        loss = nrmse(y_test, predictions, norm_value=np.ptp(X_train))
        r2 = rsquare(y_test, predictions)

        # Change the seed between instances
        variable_seed += 1

        losses.append(loss)
        r2s.append(r2)

    # Return a dictionnary of metrics. The 'loss' key is mandatory when
    # using hyperopt.
    return {'loss': np.mean(losses),
            'r2': np.mean(r2s)}